# Шаг 1. Подготовка данных для PD-модели

Что хочу сделать:
1) Загрузить датасет из data/raw.
2) Быстро глянуть структуру и целевую переменную.
3) Провалидировать данные (Pandera).
4) Разделить на train/test и сохранить в data/processed.
5) Запушить изменения в репозиторий.


In [2]:
from pathlib import Path
import pandas as pd

ROOT = Path.cwd() if Path.cwd().name == "credit" else Path.cwd().parent
RAW = ROOT / "data" / "raw" / "UCI_Credit_Card.csv"

assert RAW.exists(), f"Положи файл сюда: {RAW}"

df = pd.read_csv(RAW)
df.head(3), df.shape


(   ID  LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
 0   1    20000.0    2          2         1   24      2      2     -1     -1   
 1   2   120000.0    2          2         2   26     -1      2      0      0   
 2   3    90000.0    2          2         2   34      0      0      0      0   
 
    ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  \
 0  ...        0.0        0.0        0.0       0.0     689.0       0.0   
 1  ...     3272.0     3455.0     3261.0       0.0    1000.0    1000.0   
 2  ...    14331.0    14948.0    15549.0    1518.0    1500.0    1000.0   
 
    PAY_AMT4  PAY_AMT5  PAY_AMT6  default.payment.next.month  
 0       0.0       0.0       0.0                           1  
 1    1000.0       0.0    2000.0                           1  
 2    1000.0    1000.0    5000.0                           0  
 
 [3 rows x 25 columns],
 (30000, 25))

## Короткий EDA (без графиков)
Посмотрю столбцы, пропуски и баланс классов.


In [7]:
pd.Series(df.dtypes).head(10)


ID             int64
LIMIT_BAL    float64
SEX            int64
EDUCATION      int64
MARRIAGE       int64
AGE            int64
PAY_0          int64
PAY_2          int64
PAY_3          int64
PAY_4          int64
dtype: object

In [9]:
(df.isna().sum().sort_values(ascending=False).head(10), df.isna().sum().sum())


(ID           0
 BILL_AMT2    0
 PAY_AMT6     0
 PAY_AMT5     0
 PAY_AMT4     0
 PAY_AMT3     0
 PAY_AMT2     0
 PAY_AMT1     0
 BILL_AMT6    0
 BILL_AMT5    0
 dtype: int64,
 0)

target = "default.payment.next.month"
df[target].value_counts(), df[target].value_counts(normalize=True).round(3)


## Валидация Pandera
Проверю базовые вещи: LIMIT_BAL >= 0, SEX ∈ {1,2}, AGE в [18..100], target ∈ {0,1}.


In [14]:
import pandera as pa
from pandera import Column, DataFrameSchema, Check

schema = DataFrameSchema({
    "LIMIT_BAL": Column(pa.Int64, Check.ge(0)),
    "SEX": Column(pa.Int64, Check.isin([1, 2])),
    "EDUCATION": Column(pa.Int64),
    "MARRIAGE": Column(pa.Int64),
    "AGE": Column(pa.Int64, Check.between(18, 100)),
    "default.payment.next.month": Column(pa.Int64, Check.isin([0, 1])),
}, coerce=True)

df_valid = schema.validate(df, lazy=True)
df_valid.shape


(30000, 25)

## Train/Test split и сохранение
Сделаю стратификацию по целевой переменной.


In [21]:
from sklearn.model_selection import train_test_split

X = df_valid.drop(columns=[target])
y = df_valid[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

train = pd.concat([X_train, y_train], axis=1)
test  = pd.concat([X_test, y_test], axis=1)

PROC = ROOT / "data" / "processed"
PROC.mkdir(parents=True, exist_ok=True)
train.to_csv(PROC / "train.csv", index=False)
test.to_csv(PROC / "test.csv", index=False)

train.shape, test.shape


((24000, 25), (6000, 25))

In [ ]:
## Сохраню такой же код отдельным скриптом (для DVC)
Файл: `src/data/make_dataset.py`.
